# Deployment Export and Configuration Generation

This notebook handles the final step of the development workflow: exporting local LangGraph implementations to n8n-compatible configurations.

## Objectives
1. Export AI Agent node configurations
2. Generate Memory node configurations
3. Create workflow connection specifications
4. Generate deployment scripts
5. Validate configurations before deployment

## Setup

In [ ]:
import os
import sys
import json
from pathlib import Path
from dotenv import load_dotenv
from typing import Dict, List, Any, Optional
from datetime import datetime
import uuid

# Load environment
load_dotenv()

# Add python modules to path
project_root = Path().resolve().parent
sys.path.append(str(project_root / 'python'))

print(f"Deployment export environment ready")
print(f"Project root: {project_root}")
print(f"Current time: {datetime.now().isoformat()}")

## Configuration Templates

In [ ]:
# n8n node configuration templates
class N8nConfigTemplates:
    """Templates for generating n8n node configurations"""
    
    @staticmethod
    def ai_agent_node(system_message: str, position: List[int] = [1800, 260]) -> Dict[str, Any]:
        """Generate AI Agent node configuration"""
        return {
            "parameters": {
                "agent": "conversationalAgent",
                "model": {
                    "__rl": True,
                    "mode": "list",
                    "value": "claude-sonnet-4-20250514",
                    "cachedResultName": "Claude 4 Sonnet"
                },
                "systemMessage": system_message,
                "options": {
                    "maxIterations": 5,
                    "returnIntermediateSteps": True,
                    "temperature": 0.1
                }
            },
            "type": "@n8n/n8n-nodes-langchain.agent",
            "typeVersion": 1.8,
            "position": position,
            "id": str(uuid.uuid4()),
            "name": "AI Email Processor",
            "credentials": {
                "anthropicApi": {
                    "id": "JfNCHfdN0o9xewV4",  # Use existing credential ID
                    "name": "Anthropic account"
                }
            }
        }
    
    @staticmethod
    def memory_node(window_size: int = 5, position: List[int] = [1600, 400]) -> Dict[str, Any]:
        """Generate Memory Buffer Window node configuration"""
        return {
            "parameters": {
                "windowSize": window_size,
                "returnMessages": True,
                "inputKey": "input",
                "outputKey": "history"
            },
            "type": "@n8n/n8n-nodes-langchain.memoryBufferWindow",
            "typeVersion": 1.3,
            "position": position,
            "id": str(uuid.uuid4()),
            "name": "Email Memory"
        }
    
    @staticmethod
    def python_preprocessor_node(python_code: str, position: List[int] = [1580, 260]) -> Dict[str, Any]:
        """Generate Python preprocessing node configuration"""
        return {
            "parameters": {
                "language": "python",
                "pythonCode": python_code
            },
            "type": "n8n-nodes-base.code",
            "typeVersion": 2,
            "position": position,
            "id": str(uuid.uuid4()),
            "name": "Prepare Email Data"
        }
    
    @staticmethod
    def python_postprocessor_node(python_code: str, position: List[int] = [2020, 260]) -> Dict[str, Any]:
        """Generate Python postprocessing node configuration"""
        return {
            "parameters": {
                "language": "python",
                "pythonCode": python_code
            },
            "type": "n8n-nodes-base.code",
            "typeVersion": 2,
            "position": position,
            "id": str(uuid.uuid4()),
            "name": "Process Results"
        }

print("Configuration templates loaded")

## System Message Generation

In [ ]:
# Generate optimized system message for n8n AI Agent
def generate_system_message() -> str:
    """Generate the system message for the AI Agent node"""
    
    system_message = """
You are an intelligent email processor for Arrgh systems. Your role is to analyze incoming emails and provide structured analysis.

## Your Capabilities:
1. **Email Categorization**: Classify emails into: invoice, support, sales, general, spam
2. **Priority Assessment**: Determine urgency: high, medium, low
3. **Entity Extraction**: Extract structured data (amounts, dates, names, contacts)
4. **Action Recommendations**: Suggest appropriate next steps
5. **Reasoning**: Provide clear explanations for your analysis

## Analysis Guidelines:

**Invoice Emails**: Look for payment requests, billing information, amounts, due dates
- Keywords: invoice, bill, payment, due, amount, remittance
- Priority: Usually medium, high if overdue or urgent

**Support Emails**: Identify help requests, technical issues, questions
- Keywords: help, support, issue, problem, error, bug, question
- Priority: High if urgent/critical, medium for general support

**Sales Emails**: Detect business opportunities, proposals, meetings
- Keywords: proposal, quote, offer, deal, opportunity, meeting, demo
- Priority: Medium for opportunities, low for general inquiries

**Priority Indicators**:
- High: urgent, asap, critical, emergency, important, deadline
- Medium: follow-up, reminder, soon, this week
- Low: when convenient, no rush, general information

## Response Format:
Always respond with a JSON object containing:
```json
{
  "category": "invoice|support|sales|general|spam",
  "priority": "high|medium|low",
  "confidence": 0.95,
  "entities": {
    "amounts": ["$1,250.00"],
    "dates": ["July 23, 2025"],
    "contacts": ["john@example.com"],
    "invoice_numbers": ["INV-2025-001"],
    "keywords": ["payment", "due", "invoice"]
  },
  "suggested_actions": [
    "Forward to accounting team",
    "Add to payment tracking system",
    "Set calendar reminder for due date"
  ],
  "reasoning": "Email contains clear invoice information with specific amount and due date. Classified as medium priority invoice requiring payment processing."
}
```

## Important Notes:
- Be consistent with categorization
- Provide confidence scores between 0.0 and 1.0
- Extract all relevant entities accurately
- Suggest actionable next steps
- Keep reasoning concise but informative
- Handle incomplete or unclear emails gracefully
""".strip()
    
    return system_message

# Generate and display system message
system_message = generate_system_message()
print("=== GENERATED SYSTEM MESSAGE ===")
print(system_message)
print(f"\nLength: {len(system_message)} characters")

## Python Code Generation

In [ ]:
# Generate Python code for preprocessing and postprocessing nodes
def generate_preprocessing_code() -> str:
    """Generate Python code for email data preparation"""
    
    code = """
# Prepare email data for AI Agent processing
# Input: Parsed email content from previous node
# Output: Formatted data for AI Agent

import json
from datetime import datetime

# Get email data from previous node
email_data = $input.first().json

# Extract and clean email content
email_from = email_data.get('from', '')
email_subject = email_data.get('subject', '')
email_body = email_data.get('body', '')
email_timestamp = email_data.get('timestamp', '')
message_id = email_data.get('messageId', '')

# Generate thread ID for conversation tracking
import hashlib
thread_key = f"{email_from}-{email_subject.replace('Re:', '').replace('Fwd:', '').strip()}"
thread_id = hashlib.md5(thread_key.encode()).hexdigest()[:8]

# Prepare structured input for AI Agent
ai_input = {
    "email_data": {
        "id": message_id,
        "from": email_from,
        "subject": email_subject,
        "body": email_body,
        "timestamp": email_timestamp,
        "thread_id": thread_id
    },
    "analysis_request": {
        "categorize": True,
        "extract_entities": True,
        "assess_priority": True,
        "suggest_actions": True
    },
    "context": {
        "system": "email-processor",
        "version": "2.0",
        "processing_time": datetime.now().isoformat()
    }
}

# Log preprocessing info
print(f"Preprocessing email from {email_from}")
print(f"Subject: {email_subject}")
print(f"Thread ID: {thread_id}")

return ai_input
""".strip()
    
    return code

def generate_postprocessing_code() -> str:
    """Generate Python code for processing AI Agent results"""
    
    code = """
# Process AI Agent results and format final output
# Input: AI Agent analysis results
# Output: Formatted results for webhook response

import json
from datetime import datetime

# Get AI Agent output
ai_result = $input.first().json

# Get original email data from preprocessing node
email_data = $('Prepare Email Data').first().json.get('email_data', {})

# Parse AI response if it's a string
if isinstance(ai_result, str):
    try:
        ai_analysis = json.loads(ai_result)
    except json.JSONDecodeError:
        # Fallback if AI response is not valid JSON
        ai_analysis = {
            "category": "general",
            "priority": "low",
            "confidence": 0.5,
            "entities": {},
            "suggested_actions": ["Manual review required"],
            "reasoning": "Failed to parse AI response",
            "error": "Invalid JSON response from AI"
        }
else:
    ai_analysis = ai_result

# Validate required fields
required_fields = ['category', 'priority', 'confidence']
for field in required_fields:
    if field not in ai_analysis:
        ai_analysis[field] = 'unknown' if field != 'confidence' else 0.0

# Generate processing summary
processing_summary = {
    "email_id": email_data.get('id', 'unknown'),
    "processed_at": datetime.now().isoformat(),
    "processing_result": {
        "category": ai_analysis.get('category', 'unknown'),
        "priority": ai_analysis.get('priority', 'low'),
        "confidence": ai_analysis.get('confidence', 0.0),
        "entities": ai_analysis.get('entities', {}),
        "suggested_actions": ai_analysis.get('suggested_actions', []),
        "reasoning": ai_analysis.get('reasoning', 'No reasoning provided')
    },
    "email_metadata": {
        "from": email_data.get('from', ''),
        "subject": email_data.get('subject', ''),
        "timestamp": email_data.get('timestamp', ''),
        "thread_id": email_data.get('thread_id', '')
    },
    "processing_status": "completed",
    "system_info": {
        "processor": "email-processor-v2",
        "model": "claude-sonnet-4",
        "workflow_version": "2.0.0"
    }
}

# Add error information if present
if 'error' in ai_analysis:
    processing_summary['error'] = ai_analysis['error']
    processing_summary['processing_status'] = 'completed_with_errors'

# Log processing result
print(f"Email {processing_summary['email_id']} processed successfully")
print(f"Category: {processing_summary['processing_result']['category']}")
print(f"Priority: {processing_summary['processing_result']['priority']}")
print(f"Confidence: {processing_summary['processing_result']['confidence']}")

return processing_summary
""".strip()
    
    return code

# Generate code
preprocessing_code = generate_preprocessing_code()
postprocessing_code = generate_postprocessing_code()

print("=== GENERATED PREPROCESSING CODE ===")
print(preprocessing_code[:300] + "...")
print(f"\nPreprocessing code length: {len(preprocessing_code)} characters")

print("\n=== GENERATED POSTPROCESSING CODE ===")
print(postprocessing_code[:300] + "...")
print(f"\nPostprocessing code length: {len(postprocessing_code)} characters")

## Generate Complete Node Configurations

In [ ]:
# Generate all node configurations
def generate_all_configurations() -> Dict[str, Any]:
    """Generate all required node configurations"""
    
    # Generate individual nodes
    preprocessor_node = N8nConfigTemplates.python_preprocessor_node(
        python_code=preprocessing_code,
        position=[1580, 260]
    )
    
    ai_agent_node = N8nConfigTemplates.ai_agent_node(
        system_message=system_message,
        position=[1800, 260]
    )
    
    memory_node = N8nConfigTemplates.memory_node(
        window_size=5,
        position=[1600, 400]
    )
    
    postprocessor_node = N8nConfigTemplates.python_postprocessor_node(
        python_code=postprocessing_code,
        position=[2020, 260]
    )
    
    # Store node IDs for connection mapping
    node_ids = {
        "preprocessor": preprocessor_node["id"],
        "ai_agent": ai_agent_node["id"],
        "memory": memory_node["id"],
        "postprocessor": postprocessor_node["id"]
    }
    
    return {
        "nodes": {
            "preprocessor": preprocessor_node,
            "ai_agent": ai_agent_node,
            "memory": memory_node,
            "postprocessor": postprocessor_node
        },
        "node_ids": node_ids,
        "metadata": {
            "generated_at": datetime.now().isoformat(),
            "version": "2.0.0",
            "description": "Enhanced Arrgh Email Processor with AI Agent"
        }
    }

# Generate configurations
configurations = generate_all_configurations()

print("=== GENERATED NODE CONFIGURATIONS ===")
for node_name, node_config in configurations["nodes"].items():
    print(f"{node_name.title()}: {node_config['name']} ({node_config['type']})")
    print(f"  ID: {node_config['id']}")
    print(f"  Position: {node_config['position']}")

print(f"\nGenerated at: {configurations['metadata']['generated_at']}")
print(f"Version: {configurations['metadata']['version']}")

## Generate Node Connections

In [ ]:
# Generate node connection specifications
def generate_node_connections(node_ids: Dict[str, str]) -> Dict[str, Any]:
    """Generate connection specifications for the new nodes"""
    
    connections = {
        # Parse Email Content -> Prepare Email Data (preprocessor)
        "Parse Email Content": {
            "main": [
                [
                    {
                        "node": "Prepare Email Data",
                        "type": "main",
                        "index": 0
                    }
                ]
            ]
        },
        
        # Prepare Email Data -> AI Email Processor (agent)
        "Prepare Email Data": {
            "main": [
                [
                    {
                        "node": "AI Email Processor",
                        "type": "main",
                        "index": 0
                    }
                ]
            ]
        },
        
        # Email Memory -> AI Email Processor (memory connection)
        "Email Memory": {
            "ai_memory": [
                [
                    {
                        "node": "AI Email Processor",
                        "type": "ai_memory",
                        "index": 0
                    }
                ]
            ]
        },
        
        # AI Email Processor -> Process Results (postprocessor)
        "AI Email Processor": {
            "main": [
                [
                    {
                        "node": "Process Results",
                        "type": "main",
                        "index": 0
                    }
                ]
            ]
        },
        
        # Process Results -> Success Response
        "Process Results": {
            "main": [
                [
                    {
                        "node": "Success Response",
                        "type": "main",
                        "index": 0
                    }
                ]
            ]
        }
    }
    
    return {
        "connections": connections,
        "connection_summary": {
            "total_connections": len(connections),
            "flow_description": "Parse Email Content → Prepare Email Data → AI Email Processor (+ Email Memory) → Process Results → Success Response"
        }
    }

# Generate connections
connections_config = generate_node_connections(configurations["node_ids"])

print("=== GENERATED NODE CONNECTIONS ===")
print(f"Flow: {connections_config['connection_summary']['flow_description']}")
print(f"Total connections: {connections_config['connection_summary']['total_connections']}")

print("\n=== CONNECTION DETAILS ===")
for source_node, connections in connections_config["connections"].items():
    for connection_type, targets in connections.items():
        for target_list in targets:
            for target in target_list:
                print(f"{source_node} --{connection_type}--> {target['node']}")

## Export Complete Workflow Configuration

In [ ]:
# Create complete workflow export
def create_workflow_export() -> Dict[str, Any]:
    """Create complete workflow configuration for n8n import"""
    
    # Combine all configurations
    workflow_export = {
        "name": "Arrgh Email Processor - Enhanced",
        "nodes": list(configurations["nodes"].values()),
        "connections": connections_config["connections"],
        "active": False,  # Set to false for initial import
        "settings": {
            "executionOrder": "v1"
        },
        "staticData": None,
        "meta": {
            "templateCredsSetupCompleted": True,
            "generated_by": "n8n-development-workflow",
            "version": configurations["metadata"]["version"],
            "generated_at": configurations["metadata"]["generated_at"]
        },
        "pinData": {},
        "tags": ["email-processing", "ai-enhanced", "production"]
    }
    
    return workflow_export

# Create export
workflow_export = create_workflow_export()

print("=== WORKFLOW EXPORT SUMMARY ===")
print(f"Workflow name: {workflow_export['name']}")
print(f"Number of nodes: {len(workflow_export['nodes'])}")
print(f"Number of connections: {len(workflow_export['connections'])}")
print(f"Version: {workflow_export['meta']['version']}")
print(f"Tags: {', '.join(workflow_export['tags'])}")

# Validate export structure
required_fields = ['name', 'nodes', 'connections', 'active', 'settings']
missing_fields = [field for field in required_fields if field not in workflow_export]

if missing_fields:
    print(f"\n❌ Missing required fields: {missing_fields}")
else:
    print("\n✅ Workflow export structure is valid")

## Save Export Files

In [ ]:
# Save all export files to deploy directory
def save_export_files():
    """Save all generated configurations to files"""
    
    deploy_dir = project_root / 'deploy'
    deploy_dir.mkdir(exist_ok=True)
    
    # Save individual node configurations
    for node_name, node_config in configurations["nodes"].items():
        filename = f"{node_name}-node-config.json"
        with open(deploy_dir / filename, 'w') as f:
            json.dump(node_config, f, indent=2)
        print(f"Saved: {filename}")
    
    # Save complete workflow export
    with open(deploy_dir / 'email-processor-enhanced.json', 'w') as f:
        json.dump(workflow_export, f, indent=2)
    print("Saved: email-processor-enhanced.json")
    
    # Save connections configuration
    with open(deploy_dir / 'node-connections.json', 'w') as f:
        json.dump(connections_config, f, indent=2)
    print("Saved: node-connections.json")
    
    # Save system message separately for easy editing
    with open(deploy_dir / 'system-message.txt', 'w') as f:
        f.write(system_message)
    print("Saved: system-message.txt")
    
    # Save Python code files
    with open(deploy_dir / 'preprocessing-code.py', 'w') as f:
        f.write(preprocessing_code)
    print("Saved: preprocessing-code.py")
    
    with open(deploy_dir / 'postprocessing-code.py', 'w') as f:
        f.write(postprocessing_code)
    print("Saved: postprocessing-code.py")
    
    # Create deployment summary
    deployment_summary = {
        "export_info": {
            "generated_at": datetime.now().isoformat(),
            "version": configurations["metadata"]["version"],
            "workflow_name": workflow_export["name"]
        },
        "files_generated": [
            "email-processor-enhanced.json",
            "node-connections.json",
            "system-message.txt",
            "preprocessing-code.py",
            "postprocessing-code.py"
        ] + [f"{name}-node-config.json" for name in configurations["nodes"].keys()],
        "deployment_steps": [
            "1. Import email-processor-enhanced.json into n8n",
            "2. Verify node connections are correct",
            "3. Test with webhook-test endpoint",
            "4. Validate with real email data",
            "5. Activate workflow for production"
        ],
        "node_summary": {
            "total_nodes": len(configurations["nodes"]),
            "node_types": {
                "python_code": 2,
                "ai_agent": 1,
                "memory": 1
            }
        }
    }
    
    with open(deploy_dir / 'deployment-summary.json', 'w') as f:
        json.dump(deployment_summary, f, indent=2)
    print("Saved: deployment-summary.json")
    
    return deploy_dir, deployment_summary

# Save all files
print("=== SAVING EXPORT FILES ===")
deploy_directory, summary = save_export_files()

print(f"\n=== EXPORT COMPLETE ===")
print(f"Deploy directory: {deploy_directory}")
print(f"Files generated: {len(summary['files_generated'])}")
print(f"Ready for deployment: {summary['export_info']['workflow_name']}")

## Generate Deployment Script

In [ ]:
# Generate deployment script for n8n API
def generate_deployment_script() -> str:
    """Generate bash script for automated deployment"""
    
    script = f"""
#!/bin/bash

# Arrgh Email Processor Enhanced - Deployment Script
# Generated: {datetime.now().isoformat()}
# Version: {configurations['metadata']['version']}

set -e  # Exit on any error

# Configuration
N8N_BASE_URL="${{N8N_BASE_URL:-https://n8n.paulbonneville.com}}"
N8N_API_KEY="${{N8N_API_KEY}}"
WORKFLOW_FILE="email-processor-enhanced.json"

# Colors for output
RED='\033[0;31m'
GREEN='\033[0;32m'
YELLOW='\033[1;33m'
NC='\033[0m' # No Color

echo -e "${{GREEN}}=== Arrgh Email Processor Enhanced Deployment ===${{NC}}"
echo "Deploying to: $N8N_BASE_URL"
echo "Workflow file: $WORKFLOW_FILE"
echo

# Check if API key is set
if [ -z "$N8N_API_KEY" ]; then
    echo -e "${{RED}}Error: N8N_API_KEY environment variable not set${{NC}}"
    echo "Please set your n8n API key:"
    echo "export N8N_API_KEY=your_api_key_here"
    exit 1
fi

# Check if workflow file exists
if [ ! -f "$WORKFLOW_FILE" ]; then
    echo -e "${{RED}}Error: Workflow file $WORKFLOW_FILE not found${{NC}}"
    echo "Please run the deployment export notebook first"
    exit 1
fi

# Function to make API calls
api_call() {{
    local method=$1
    local endpoint=$2
    local data=$3
    
    curl -s -X "$method" \
        "$N8N_BASE_URL/api/v1/$endpoint" \
        -H "Authorization: Bearer $N8N_API_KEY" \
        -H "Content-Type: application/json" \
        -d "$data"
}}

# Step 1: Check if workflow already exists
echo -e "${{YELLOW}}Step 1: Checking existing workflows...${{NC}}"
existing_workflows=$(api_call "GET" "workflows" "")

workflow_id=$(echo "$existing_workflows" | jq -r '.data[] | select(.name == "Arrgh Email Processor - Enhanced") | .id')

if [ "$workflow_id" != "null" ] && [ -n "$workflow_id" ]; then
    echo "Found existing workflow with ID: $workflow_id"
    echo -e "${{YELLOW}}Updating existing workflow...${{NC}}"
    
    # Update existing workflow
    result=$(api_call "PUT" "workflows/$workflow_id" "$(cat $WORKFLOW_FILE)")
    
    if echo "$result" | jq -e '.id' > /dev/null; then
        echo -e "${{GREEN}}✓ Workflow updated successfully${{NC}}"
    else
        echo -e "${{RED}}✗ Failed to update workflow${{NC}}"
        echo "Error: $result"
        exit 1
    fi
else
    echo "No existing workflow found"
    echo -e "${{YELLOW}}Creating new workflow...${{NC}}"
    
    # Create new workflow
    result=$(api_call "POST" "workflows" "$(cat $WORKFLOW_FILE)")
    
    if echo "$result" | jq -e '.id' > /dev/null; then
        workflow_id=$(echo "$result" | jq -r '.id')
        echo -e "${{GREEN}}✓ Workflow created successfully with ID: $workflow_id${{NC}}"
    else
        echo -e "${{RED}}✗ Failed to create workflow${{NC}}"
        echo "Error: $result"
        exit 1
    fi
fi

# Step 2: Test workflow (optional)
echo -e "${{YELLOW}}Step 2: Testing workflow...${{NC}}"
echo "You can test the workflow using:"
echo "./test-sns-webhook.sh test"

# Step 3: Activation prompt
echo -e "${{YELLOW}}Step 3: Workflow activation${{NC}}"
echo "Workflow deployed but not activated"
echo "To activate, run:"
echo "curl -X POST '$N8N_BASE_URL/api/v1/workflows/$workflow_id/activate' -H 'Authorization: Bearer $N8N_API_KEY'"

echo
echo -e "${{GREEN}}=== Deployment Complete ===${{NC}}"
echo "Workflow ID: $workflow_id"
echo "Status: Deployed (inactive)"
echo "Next steps:"
echo "1. Test the workflow with test data"
echo "2. Verify all nodes are working correctly"
echo "3. Activate the workflow for production use"
""".strip()
    
    return script

# Generate and save deployment script
deployment_script = generate_deployment_script()

with open(deploy_directory / 'deploy.sh', 'w') as f:
    f.write(deployment_script)

# Make script executable
import stat
script_path = deploy_directory / 'deploy.sh'
script_path.chmod(script_path.stat().st_mode | stat.S_IEXEC)

print("=== DEPLOYMENT SCRIPT GENERATED ===")
print(f"Script saved: {script_path}")
print("\nTo deploy, run:")
print(f"cd {deploy_directory}")
print("export N8N_API_KEY=your_api_key_here")
print("./deploy.sh")

print("\nScript preview:")
print(deployment_script[:500] + "...")

## Final Validation and Summary

In [ ]:
# Perform final validation of all exports
def final_validation() -> Dict[str, bool]:
    """Perform comprehensive validation of all generated files"""
    
    validation_results = {}
    
    # Check file existence
    required_files = [
        'email-processor-enhanced.json',
        'deployment-summary.json',
        'deploy.sh',
        'system-message.txt',
        'preprocessing-code.py',
        'postprocessing-code.py'
    ]
    
    for filename in required_files:
        file_path = deploy_directory / filename
        validation_results[f"file_exists_{filename}"] = file_path.exists()
    
    # Validate JSON files
    json_files = ['email-processor-enhanced.json', 'deployment-summary.json']
    for filename in json_files:
        try:
            with open(deploy_directory / filename, 'r') as f:
                json.load(f)
            validation_results[f"valid_json_{filename}"] = True
        except (json.JSONDecodeError, FileNotFoundError):
            validation_results[f"valid_json_{filename}"] = False
    
    # Validate workflow structure
    try:
        required_workflow_fields = ['name', 'nodes', 'connections', 'active', 'settings']
        missing_fields = [field for field in required_workflow_fields if field not in workflow_export]
        validation_results['workflow_structure_valid'] = len(missing_fields) == 0
    except:
        validation_results['workflow_structure_valid'] = False
    
    # Validate node configurations
    node_validation = True
    for node_name, node_config in configurations["nodes"].items():
        required_node_fields = ['id', 'name', 'type', 'parameters', 'position']
        if not all(field in node_config for field in required_node_fields):
            node_validation = False
            break
    
    validation_results['all_nodes_valid'] = node_validation
    
    # Validate system message length
    validation_results['system_message_appropriate_length'] = 100 < len(system_message) < 5000
    
    # Validate Python code
    validation_results['preprocessing_code_not_empty'] = len(preprocessing_code.strip()) > 0
    validation_results['postprocessing_code_not_empty'] = len(postprocessing_code.strip()) > 0
    
    return validation_results

# Run final validation
validation_results = final_validation()

print("=== FINAL VALIDATION RESULTS ===")
all_passed = True
for check, passed in validation_results.items():
    status = "✅ PASS" if passed else "❌ FAIL"
    print(f"{status} - {check.replace('_', ' ').title()}")
    if not passed:
        all_passed = False

print(f"\n=== OVERALL VALIDATION: {'✅ ALL CHECKS PASSED' if all_passed else '❌ SOME CHECKS FAILED'} ===")

if all_passed:
    print("\n🚀 READY FOR DEPLOYMENT!")
    print("\nNext steps:")
    print("1. Review generated files in deploy/ directory")
    print("2. Set N8N_API_KEY environment variable")
    print("3. Run ./deploy.sh to deploy to n8n")
    print("4. Test with webhook-test endpoint")
    print("5. Activate workflow for production")
else:
    print("\n⚠️  Please fix validation errors before deployment")

print(f"\n=== EXPORT SUMMARY ===")
print(f"Workflow: {workflow_export['name']}")
print(f"Version: {configurations['metadata']['version']}")
print(f"Nodes: {len(workflow_export['nodes'])}")
print(f"Files generated: {len(summary['files_generated'])}")
print(f"Deploy directory: {deploy_directory}")
print(f"Generated at: {configurations['metadata']['generated_at']}")